In [1]:
from utils_req import *

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import datajoint as dj
dj.config['database.host'] = 'at-database.ad.bcm.edu'
dj.config['database.user'] = 'raj'
dj.config['database.password'] = 'language-wonderful-joy-system'
dj.conn()

from stimulus import stimulus
from neuro_data.movies import data_schemas
from neuro_data.movies.stats import load_dataset

Connecting raj@at-database.ad.bcm.edu:3306


/Users/rajdbz/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def GetDimensionality(X):
    covX = np.cov(X) # rows of X are variables, columns are data points
    E = np.linalg.eigvals(covX) #compute eigenvalues of the covariance
    return np.sum(E)**2/np.sum(E**2)

In [3]:
"""
Pick dataset
"""
dataset = 9

In [4]:
"""
Load the data set: movies, neural responses, etc
"""

groupidstr = 'group_id=' + str(dataset)
key = (data_schemas.MovieMultiDataset.Member & groupidstr).fetch('KEY')

dset = load_dataset(key)
key

[{'animal_id': 16314,
  'group_id': 9,
  'pipe_version': 1,
  'preproc_id': 0,
  'scan_idx': 1,
  'segmentation_method': 3,
  'session': 3,
  'spike_method': 5}]

In [5]:
MonetId = np.nonzero((dset.types == 'stimulus.Monet2')*1)[0]
TrippyId = np.nonzero((dset.types == 'stimulus.Trippy')*1)[0]
NaturalId = np.nonzero((dset.types == 'stimulus.Clip')*1)[0]

In [14]:
C_hashes = dset.condition_hashes[NaturalId]
C_hashes_unique, UniqueId = np.unique(C_hashes,return_index=True)

In [18]:
mdict = {'UniqueId':UniqueId+1}
savemat('./DimensionalityAnalysis/NaturalClipIndices_DataSet9.mat',mdict)

In [19]:
x = np.load('./DimensionalityAnalysis/Inputs/Natural_dataset9.npy')

In [21]:
x = x[:,UniqueId,0,:,:]

In [24]:
mdict = {'NaturalClips':x}
savemat('./DimensionalityAnalysis/NaturalClips_DS9.mat',mdict)

In [6]:
"""
Create movies clips
"""

MonetMovies = []
for k in tqdm(range(len(MonetId))):
    MonetMovies.append(dset[MonetId[k]].inputs)

MonetMovies =  np.asarray(MonetMovies).transpose(2,0,1,3,4)

TrippyMovies = []
for k in tqdm(range(len(TrippyId))):
    TrippyMovies.append(dset[TrippyId[k]].inputs)

TrippyMovies =  np.asarray(TrippyMovies).transpose(2,0,1,3,4)

NaturalMovies = []
for k in tqdm(range(len(NaturalId))):
    NaturalMovies.append(dset[NaturalId[k]].inputs)

NaturalMovies =  np.asarray(NaturalMovies).transpose(2,0,1,3,4)

100%|██████████| 372/372 [00:08<00:00, 44.65it/s]


In [7]:
print(MonetMovies.shape)
print(TrippyMovies.shape)
print(NaturalMovies.shape)

(450, 38, 1, 36, 64)
(450, 37, 1, 36, 64)
(301, 372, 1, 36, 64)


In [8]:
"""
Save the movies
"""

fname = './DimensionalityAnalysis/Inputs/Monet_dataset' + str(dataset)
np.save(fname,MonetMovies)
fname = './DimensionalityAnalysis/Inputs/Trippy_dataset' + str(dataset)
np.save(fname,TrippyMovies)
fname = './DimensionalityAnalysis/Inputs/Natural_dataset' + str(dataset)
np.save(fname,NaturalMovies)

In [9]:
"""
Reshape the movies for dimensionality analysis
"""
MonetMovies = MonetMovies.squeeze().transpose(2,3,0,1).reshape(36*64,-1)
TrippyMovies = TrippyMovies.squeeze().transpose(2,3,0,1).reshape(36*64,-1)
NaturalMovies = NaturalMovies.squeeze().transpose(2,3,0,1).reshape(36*64,-1)

In [10]:
print(MonetMovies.shape)
print(TrippyMovies.shape)
print(NaturalMovies.shape)

(2304, 17100)
(2304, 16650)
(2304, 111972)


In [11]:
dim_M = GetDimensionality(MonetMovies)
dim_T = GetDimensionality(TrippyMovies)
dim_N = GetDimensionality(NaturalMovies)

print(dim_M)
print(dim_T)
print(dim_N)

372.36949483528446
343.85996774576284
4.151202939619348


In [12]:
"""
Save the reshaped movies as mat file
"""
mdict = {'MonetMovies':MonetMovies, 'TrippyMovies':TrippyMovies, 'NaturalMovies':NaturalMovies}
fname = './DimensionalityAnalysis/Inputs/Movies' + str(dataset) + '.mat'
savemat(fname, mdict)

In [13]:
"""
delete variables
"""
del MonetMovies, TrippyMovies, NaturalMovies, mdict

In [6]:
"""
Extract the neural Responses
"""

MonetResponses = []
for k in tqdm(range(len(MonetId))):
    MonetResponses.append(dset[MonetId[k]].responses)

MonetResponses =  np.asarray(MonetResponses).transpose(2,0,1)

TrippyResponses = []
for k in tqdm(range(len(TrippyId))):
    TrippyResponses.append(dset[TrippyId[k]].responses)

TrippyResponses =  np.asarray(TrippyResponses).transpose(2,0,1)

NaturalResponses = []
for k in tqdm(range(len(NaturalId))):
    NaturalResponses.append(dset[NaturalId[k]].responses)

NaturalResponses =  np.asarray(NaturalResponses).transpose(2,0,1)

100%|██████████| 287/287 [00:07<00:00, 36.93it/s]


In [7]:
"""
Retain only responses of neurons in V1 L2/3
"""
V1Id = np.nonzero(dset.neurons.area == 'V1')[0]
L23Id = np.nonzero(dset.neurons.layer == 'L2/3')[0]
NIdx = np.intersect1d(V1Id,L23Id)

MonetResponses = MonetResponses[NIdx]
TrippyResponses = TrippyResponses[NIdx]
NaturalResponses = NaturalResponses[NIdx]

In [8]:
print(MonetResponses.shape)
print(TrippyResponses.shape)
print(NaturalResponses.shape)

(5701, 143, 300)
(5701, 138, 300)
(5701, 287, 301)


In [9]:
"""
Save the responses
"""
fname = './DimensionalityAnalysis/NeuralResponses/Monet_dataset' + str(dataset)
np.save(fname,MonetResponses)
fname = './DimensionalityAnalysis/NeuralResponses/Trippy_dataset' + str(dataset)
np.save(fname,TrippyResponses)
fname = './DimensionalityAnalysis/NeuralResponses/Natural_dataset' + str(dataset)
np.save(fname,NaturalResponses)



In [10]:
"""
Save responses as mat files
"""
mdict = {'MonetResponses':MonetResponses}
fname = './DimensionalityAnalysis/NeuralResponses/Monet_dataset' + str(dataset) + '.mat'
savemat(fname,mdict)

mdict = {'TrippyResponses':TrippyResponses}
fname = './DimensionalityAnalysis/NeuralResponses/Trippy_dataset' + str(dataset) + '.mat'
savemat(fname,mdict)

mdict = {'NaturalResponses':NaturalResponses}
fname = './DimensionalityAnalysis/NeuralResponses/Natural_dataset' + str(dataset) + '.mat'
savemat(fname,mdict)

In [56]:
"""
delete variables
"""
del MonetResponses, TrippyResponses, NaturalResponses, mdict

In [68]:
# Cx = np.cov(X) # each row corresponds to a variables .. columns to data points for numpy cov
# Ex = np.linalg.eigvals(Cx)
# dimx = np.sum(Ex)**2/np.sum(np.abs(Ex**2))
# print(dimx)

In [7]:
# responses = StandardScaler().fit_transform(responses)
# pca = PCA(.99)
# pca.fit(responses)
# np.sum(pca.explained_variance_)